In [13]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords

In [14]:
# https://finance.naver.com/news/mainnews.nhn?date=2020-02-07&page=2
url = 'https://finance.naver.com/news/mainnews.nhn'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [15]:
headline = []
for index, link in enumerate(soup.findAll("a", href=re.compile("^/news/news_read.nhn."))):
    if 'href' in link.attrs and link.get('title') != None:
        headline.append(link.get('title'))

In [41]:
def get_headline(date):
    headline = []
    for page in [1, 2]:
        url = f'{url}?date={date}&page={page}'
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        for index, link in enumerate(soup.findAll("a", href=re.compile("^/news/news_read.nhn."))):
            if 'href' in link.attrs and link.get('title') != None:
                headline.append(link.get('title'))
    return list(set(headline))

In [36]:
()

['서울 땅값 7.8% ↑…비강남권도 대폭 상승',
 '"체온조차 안 재요"…건설현장 코로나19 \'무방비\'',
 'EY한영 임시대표에 박용근 감사본부장',
 '기생충 아카데미 4관왕에 기생한 테마주?',
 '삼성 신형 갤럭시 S11 아닌 S20…경쟁사와 한판승부 예고',
 '증선위, 최대주주 차명주식 누락 태광산업에 과징금',
 '12일 장 마감 후 주요 종목뉴스',
 '[시그널] 실적악화-신용등급하락···LGD, 회사채 4,100억 상환 어쩌나',
 '[뉴욕마감] 파월 의장 발언에 뉴욕증시 또 사상최고치',
 '[유럽증시]신종코로나 퇴조 기대에 여행주 급등…상승',
 '유해균 99.999% 제거… 분주한 LG디오스 식기세척기 생산현장',
 '취업자 증가 50만 명 넘었지만…90%는 60대 이상',
 '[Asia마감]"시장의 시선은 이제 코로나→개별 기업"',
 '"200兆 해외 게임시장 잡자" 코스닥行 노리는 팡스카이',
 '[마켓뷰] 코스피, 기관·개인 ‘사자’에 2230선…우한 폐렴 반등 국면',
 '[유럽개장]신종코로나 공포 속 상승 출발…독일 0.31% ↑',
 '2층짜리 마라탕집 \'텅텅\'…소상공인 98% "매출 떨어졌다"']

In [17]:
# removing functuation
def remove_punctuation(text):
    import string
    # replacing the puntuations with no space,
    # which in effect deletes the punctuation marks
    translator = str.maketrans(' ', ' ', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

In [18]:
# extract the stop words
# extracting the stopwords
sw = stopwords.words()
print(f'Number of stopwords: {len(sw)}')
stopwords_ko = [line.replace('\n', '') for line in open('./meta/stopwords_ko', 'r', encoding='UTF8').readlines()]
print(f'Number of stopwords_ko: {len(stopwords_ko)}')
stopwords_total = stopwords_ko + sw
print(f'Number of stopwords_total: {len(stopwords_total)}')
# removing stopwords
def remove_stopwords(text):
    # removing the stop words and lowercasing the selected words
    text = [word for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

Number of stopwords: 6800
Number of stopwords_ko: 675
Number of stopwords_total: 7475


In [19]:
evaluation = pd.DataFrame()
evaluation['headline'] = headline
evaluation['tokenization'] = [re.sub(r'[^A-Za-z0-9가-힣一-龥]', ' ', i) for i in evaluation.headline]
evaluation['tokenization'] = evaluation['tokenization'].apply(remove_punctuation).apply(remove_stopwords)
evaluation['tokenization'] = [i.split(' ') for i in evaluation['tokenization']]

In [20]:
evaluation

,headline,tokenization
0,"증선위, 최대주주 차명주식 누락 태광산업에 과징금","[증선위, 최대주주, 차명주식, 누락, 태광산업에, 과징금]"
1,"[시그널] 실적악화-신용등급하락···LGD, 회사채 4,100억 상환 어쩌나","[시그널, 실적악화, 신용등급하락, LGD, 회사채, 4, 100억, 상환, 어쩌나]"
2,서울 땅값 7.8% ↑…비강남권도 대폭 상승,"[서울, 땅값, 7, 8, 비강남권도, 대폭, 상승]"
3,취업자 증가 50만 명 넘었지만…90%는 60대 이상,"[취업자, 증가, 50만, 명, 넘었지만, 90, 는, 60대, 이상]"
4,삼성 신형 갤럭시 S11 아닌 S20…경쟁사와 한판승부 예고,"[삼성, 신형, 갤럭시, S11, 아닌, S20, 경쟁사와, 한판승부, 예고]"
5,EY한영 임시대표에 박용근 감사본부장,"[EY한영, 임시대표에, 박용근, 감사본부장]"
6,"""200兆 해외 게임시장 잡자"" 코스닥行 노리는 팡스카이","[200兆, 해외, 게임시장, 잡자, 코스닥行, 노리는, 팡스카이]"
7,유해균 99.999% 제거… 분주한 LG디오스 식기세척기 생산현장,"[유해균, 99, 999, 제거, 분주한, LG디오스, 식기세척기, 생산현장]"
8,"2층짜리 마라탕집 '텅텅'…소상공인 98% ""매출 떨어졌다""","[2층짜리, 마라탕집, 텅텅, 소상공인, 98, 매출, 떨어졌다]"
9,"""체온조차 안 재요""…건설현장 코로나19 '무방비'","[체온조차, 안, 재요, 건설현장, 코로나19, 무방비]"


In [21]:
negative = [line.replace('\n', '') for line in open('./meta/negative', 'r', encoding='UTF8').readlines()]
positive = [line.replace('\n', '') for line in open('./meta/positive', 'r', encoding='UTF8').readlines()]
sentimental = pd.read_csv('./meta/sentimental', sep='\t')

In [22]:
negative[:5], positive[:5]

(['가난', '가난뱅이', '가난살이', '가난살이하다', '가난설음'],
 ['(-;', '(^^)', '(^-^)', '(^^*', '(^_^)'])

In [23]:
sentimental.head(5)

,keyword,score
0,(-;,1
1,(;_;),-1
2,(^^),1
3,(^-^),1
4,(^^*,1


In [24]:
evaluation.head()

,headline,tokenization
0,"증선위, 최대주주 차명주식 누락 태광산업에 과징금","[증선위, 최대주주, 차명주식, 누락, 태광산업에, 과징금]"
1,"[시그널] 실적악화-신용등급하락···LGD, 회사채 4,100억 상환 어쩌나","[시그널, 실적악화, 신용등급하락, LGD, 회사채, 4, 100억, 상환, 어쩌나]"
2,서울 땅값 7.8% ↑…비강남권도 대폭 상승,"[서울, 땅값, 7, 8, 비강남권도, 대폭, 상승]"
3,취업자 증가 50만 명 넘었지만…90%는 60대 이상,"[취업자, 증가, 50만, 명, 넘었지만, 90, 는, 60대, 이상]"
4,삼성 신형 갤럭시 S11 아닌 S20…경쟁사와 한판승부 예고,"[삼성, 신형, 갤럭시, S11, 아닌, S20, 경쟁사와, 한판승부, 예고]"


In [25]:
evaluation['positive_score'] = 0
evaluation['negative_score'] = 0

In [26]:
evaluation

,headline,tokenization,positive_score,negative_score
0,"증선위, 최대주주 차명주식 누락 태광산업에 과징금","[증선위, 최대주주, 차명주식, 누락, 태광산업에, 과징금]",0,0
1,"[시그널] 실적악화-신용등급하락···LGD, 회사채 4,100억 상환 어쩌나","[시그널, 실적악화, 신용등급하락, LGD, 회사채, 4, 100억, 상환, 어쩌나]",0,0
2,서울 땅값 7.8% ↑…비강남권도 대폭 상승,"[서울, 땅값, 7, 8, 비강남권도, 대폭, 상승]",0,0
3,취업자 증가 50만 명 넘었지만…90%는 60대 이상,"[취업자, 증가, 50만, 명, 넘었지만, 90, 는, 60대, 이상]",0,0
4,삼성 신형 갤럭시 S11 아닌 S20…경쟁사와 한판승부 예고,"[삼성, 신형, 갤럭시, S11, 아닌, S20, 경쟁사와, 한판승부, 예고]",0,0
5,EY한영 임시대표에 박용근 감사본부장,"[EY한영, 임시대표에, 박용근, 감사본부장]",0,0
6,"""200兆 해외 게임시장 잡자"" 코스닥行 노리는 팡스카이","[200兆, 해외, 게임시장, 잡자, 코스닥行, 노리는, 팡스카이]",0,0
7,유해균 99.999% 제거… 분주한 LG디오스 식기세척기 생산현장,"[유해균, 99, 999, 제거, 분주한, LG디오스, 식기세척기, 생산현장]",0,0
8,"2층짜리 마라탕집 '텅텅'…소상공인 98% ""매출 떨어졌다""","[2층짜리, 마라탕집, 텅텅, 소상공인, 98, 매출, 떨어졌다]",0,0
9,"""체온조차 안 재요""…건설현장 코로나19 '무방비'","[체온조차, 안, 재요, 건설현장, 코로나19, 무방비]",0,0


In [15]:
positive_score, negative_score = [], []
p_dict = ['↑', '급상승', '상승', '올랐다', '오름세', '호재', '안착', '대박', '기회', '잡았나', '출발', '반등', '사자', '급등', '승부', '증가', '최고치']
n_dict = ['↓', '상장폐지', '뚝', '관리종목', '주의보', '낚시', '우려', '둔화', '공포', '사임', '어렵다', '어려운 ', '썰렁', '손실', '적자', '과징금', '하락', '약화']
stopword = ['이다', '너무', '은', '는', '이', '가', '을', '를', '에', '게', , ]

for idx, row in evaluation.iterrows():
    p, n = 0, 0
    for pos in p_dict:
        if pos in row.headline:
            p += 1
            print(pos)
    for neg in n_dict:
        if neg in row.headline:
            n += 1
            print(neg)
    positive_score.append(p)
    negative_score.append(n)

↑
상승
공포


In [16]:
positive_score

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [17]:
negative_score

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]